In [3]:
import pandas as pd
import os
from astropy.io import fits
import glob
import astroalign as aa

In [11]:
df = pd.read_csv('startimes.csv')

In [5]:
df

,Unnamed: 0,file,time
0,77,Reduced_star_Light_Blue_45_secs_001.fits,2023-01-23 18:45:01
1,78,Reduced_star_Light_Blue_45_secs_002.fits,2023-01-23 18:48:07
2,79,Reduced_star_Light_Blue_45_secs_003.fits,2023-01-23 18:51:14
3,80,Reduced_star_Light_Blue_45_secs_004.fits,2023-01-23 18:54:19
4,81,Reduced_star_Light_Blue_45_secs_005.fits,2023-01-23 18:57:12
...,...,...,...
107,72,Reduced_star_Light_Blue_30_secs_073.fits,2023-01-23 23:54:22
108,73,Reduced_star_Light_Blue_30_secs_074.fits,2023-01-23 23:56:34
109,74,Reduced_star_Light_Blue_30_secs_075.fits,2023-01-23 23:58:46
110,75,Reduced_star_Light_Blue_30_secs_076.fits,2023-01-24 00:00:58


In [6]:
filepathfits = '/Users/maxrosenblatt/Desktop/AstroData/2023_1_23/Light/Blue/Reduced/'

In [7]:
print(filepathfits+df['file'][0])

/Users/maxrosenblatt/Desktop/AstroData/2023_1_23/Light/Blue/Reduced/Reduced_star_Light_Blue_45_secs_001.fits


In [9]:
filesInOrder = []

for i in range(len(df['file'])):
    filesInOrder.append(filepathfits+df['file'][i])


refimage = filesInOrder[0]
with fits.open(refimage) as hdul:
    refimage_data = hdul[0].data.astype('<f4')
    refheader = hdul[0].header.copy()


i = 0
for file in filesInOrder[1:]:
    with fits.open(file) as hdul:
        image_data = hdul[0].data.astype('<f4')
        aligned_image, footprint = aa.register(target=refimage_data, source=image_data, max_control_points=3)
        aligned_hdul = fits.PrimaryHDU(aligned_image, header=refheader)
        aligned_hdul.header.set('ORIGFILE', os.path.basename(file), 'Original filename')
        aligned_hdul.header.set('ORIGTIME', hdul[0].header.get('DATE-OBS'), 'Original observation time')
        aligned_hdul.writeto(filepathfits+'aligned_'+(file.replace(filepathfits, "")), overwrite=True)
        i += 1


KeyboardInterrupt: 